In [8]:
import torch
import pickle
from src.adversarial_attacks_on_feature_dependent_graphs.data import *



In [ ]:
path = '../../../shared/modelnet/modelnet40_normal_resampled/modelnet40_train_1024pts_fps.dat'
# x = torch.load(path)
with open(path, 'rb') as f:
    list_of_points, list_of_labels = pickle.load(f)

In [1]:
sys.path.append('/nfs/homedirs/bindera/Adversarial Attacks on Feature-Dependent Graphs/src/adversarial_attacks_on_feature_dependent_graphs')

In [6]:
len(list_of_points)

9843

In [9]:
data_folder = '../../../shared/modelnet/modelnet40_normal_resampled'
data_train, data_val, data_test = get_dataset(name = 'modelnet40', data_folder = data_folder)

The size of train data is 9843
Load processed data from ../../../shared/modelnet/modelnet40_normal_resampled/modelnet40_train_1024pts_fps.dat...
The size of test data is 2468
Load processed data from ../../../shared/modelnet/modelnet40_normal_resampled/modelnet40_test_1024pts_fps.dat...


In [41]:
def to_tensor(dataset):
    xs , ys = [], []

    for x, y in dataset:
        xs.append(torch.tensor(x))
        ys.append(torch.tensor(y))
    
    return torch.stack(xs), torch.stack(ys)

In [57]:
x,y = to_tensor(data_test)

In [58]:
x.shape

torch.Size([2468, 1024, 3])

In [23]:
y

tensor(39, dtype=torch.int32)

In [33]:
t = torch.tensor(4)

In [40]:
torch.stack([t,t])

tensor([4, 4])

In [60]:
torch.Tensor([[1,1], [1,2], [4,5]]).shape

torch.Size([3, 2])

In [61]:
import torch
from torch_geometric.nn import radius_graph

# x = torch.Tensor([[-1, -1], [-1, 1], [1, -1], [1, 1]])
# x = torch.Tensor([[1,1], [1,2], [4,5]])
batch = torch.tensor([0, 1, 0])
edge_index = radius_graph(x[0], r=1)
edge_index

tensor([[ 493,   34,  126,  ..., 1003,  161,  484],
        [   0,    0,    0,  ..., 1023, 1023, 1023]])

In [62]:
import os.path as osp

import torch
from torch_geometric.data import Dataset, download_url, InMemoryDataset
from torch_geometric.data import Data

import torch
from torch_geometric.nn import radius_graph



class PointCloudDataset(InMemoryDataset):  # Dataset
    def __init__(self, data_folder = '../../../shared/modelnet/modelnet40_normal_resampled', split = 'train', transform=None, pre_transform=None, pre_filter=None):
        super().__init__(None, transform, pre_transform, pre_filter)

        subset = self.load(data_folder=data_folder, split=split)

        self.data_list = self.extract_data_from_dict(subset)

        
    def load(self, data_folder, split):
        
        data_train, data_val, data_test = get_dataset(name = 'modelnet40', data_folder = data_folder)

        if split == 'train':
            return data_train
        elif split == 'val':
            return data_val
        elif split == 'test':
            return data_test
        else:
            raise KeyError

    def to_tensor(self, dataset):
        xs , ys = [], []

        for x, y in dataset:
            xs.append(torch.tensor(x))
            ys.append(torch.tensor(y))
        
        return torch.stack(xs), torch.stack(ys)

    def len(self):
        return len(self.data_list)

    def get(self, idx):
        data = self.data_list[idx]
        return data

    def extract_data_from_dict(self, subset):
        
        # subset = self.to_tensor(subset)

        list_of_data_objects = []

        i = 0

        for x, y in subset:
          
            x = torch.tensor(x)

            # x = torch.Tensor([[-1, -1], [-1, 1], [1, -1], [1, 1]])
            # x = torch.Tensor([[1,1], [1,2], [4,5]])
            
            edge_index = radius_graph(x=x, r=1)
            data = Data(x = x, 
                        edge_index = edge_index.t().contiguous(), 
                        y = torch.tensor([y], dtype=torch.long))

            list_of_data_objects.append(data)

        return list_of_data_objects

In [63]:
dataset = PointCloudDataset()

The size of train data is 9843
Load processed data from ../../../shared/modelnet/modelnet40_normal_resampled/modelnet40_train_1024pts_fps.dat...
The size of test data is 2468
Load processed data from ../../../shared/modelnet/modelnet40_normal_resampled/modelnet40_test_1024pts_fps.dat...


In [65]:
dataset[0]

Data(x=[1024, 3], edge_index=[32768, 2], y=[1])

In [1]:
from src.adversarial_attacks_on_feature_dependent_graphs.dataset import PointCloudDataset
from src.adversarial_attacks_on_feature_dependent_graphs.data import *

val_dataset = PointCloudDataset(split='val')

/nfs/homedirs/bindera/miniconda3/envs/test_torch/lib/python3.7/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


The size of train data is 9843
Load processed data from ../../../shared/modelnet/modelnet40_normal_resampled/modelnet40_train_1024pts_fps.dat...
The size of test data is 2468
Load processed data from ../../../shared/modelnet/modelnet40_normal_resampled/modelnet40_test_1024pts_fps.dat...


100%|██████████| 1968/1968 [01:01<00:00, 31.86it/s]


In [2]:
path_to_data = '/nfs/students/bindera/data/modelnet_pyg/val_dataset.pt'
torch.save(val_dataset, path_to_data)